In [1]:
def Previsaodotempo(Tmax,Tmin,Umi):
    if Tmax > Tmin:
        return True
    else:
        return False

In [2]:
import tkinter as Tkinter
import random
 
class minhaApp_tk(Tkinter.Tk):

    def __init__(self,parent):
        Tkinter.Tk.__init__(self,parent)
        self.parent = parent
        self.initialize()
        
         
         
    def initialize(self):     
        self.grid()            
        
        self.instruLabel = Tkinter.Label(self,text="Insira os dados abaixo para descobrir se choverá amanhã", height = 2, font=("Helvetica", 10))
        self.instruLabel.grid(columnspan=3,column=0,row=0,sticky='EW') 

        
        self.lblC= Tkinter.Label(self, text="Temperatura Máxima")     
        self.lblC.grid(column=0, row=1)              
         
        self.lblSep= Tkinter.Label(self, text="Temperatura Mínima")     
        self.lblSep.grid(column=1, row=1)               
 
 
        self.lblF= Tkinter.Label(self, text="Média de Umidade entre ontem e hoje")     
        self.lblF.grid(column=2, row=1)               
 
        self.entrymax = Tkinter.Entry(self)           
        self.entrymax.grid(column=0,row=2,sticky='EW')
        
        self.entrymin = Tkinter.Entry(self)           
        self.entrymin.grid(column=1,row=2,sticky='EW')
 
        self.entryumi = Tkinter.Entry(self)           
        self.entryumi.grid(column=2,row=2,sticky='EW')
 
 
 
        self.btnCalculaF = Tkinter.Button(self,text=u"Prever o tempo",command=self.OnButtonCalculaChuva)  
                                                                   
        self.btnCalculaF.grid(column=1,row=3) 
        
        self.resuLabel = Tkinter.Label(self,text=" ", height = 2, font=("Helvetica", 28))
        self.resuLabel.grid(columnspan=3,column=0,row=5,sticky='EW') 

    def OnButtonCalculaChuva(self):     
        Tmax= float(self.entrymax.get())
        Tmin= float(self.entrymin.get()) 
        Umi= float(self.entryumi.get())  
        resultado = Previsaodotempo(Tmax,Tmin,Umi)
        if resultado == True:
            resultado = 'VAI CHOVER'
        else:
            resultado = 'NÃO VAI CHOVER'
        self.resuLabel.config(text=resultado)
 


if __name__ == "__main__":
    app = minhaApp_tk(None)     
    app.title('Choverá amanha?')
    app.mainloop()              